## 0. 今日の量子コンピュータの問題

- Noisy Intermediate-Scale Quantum (NISQ) デバイス
    - 量子回路が深くなる（ゲート数が多くなる）ほど、誤差が大きくなる
    - 十分な量子ビット数ではない
- 量子デバイスは特別なゲート演算のみが用意されている
- 特定のqubits間の量子ビット演算(multi qubit operation)しか用意されていない
- それぞれの量子デバイスに対して、量子ソフトウェアツールキットが用意されてる


### 0-1. TKETとは
- Quantum Software Development Kit
- C++で実装
- pythonモジュール　`pytket`で利用可能
- 最適化コンパイラ：　ユーザーフレンドリーな回路→量子デバイスで実行可能な回路に変換可能
    - Language-agnostic (多くの量子プログラミングフレームワーク(qiskit, Cirq, etc)をサポート)
    - Retagetable (多くの量子デバイス(IBM, Honeywell, etc)をサポート)
    - Circuit Optimisation (量子計算時に生じるデバイスエラーの影響を最小化。デバイス依存＆デバイス非依存のものが実装)
    
<img src="./fig/tket1.png" width="750">



#### 参照
- [pytket API ドキュメント](https://docs.quantinuum.com/tket/api-docs/index.html)
- [pytket ユーザーガイド](https://docs.quantinuum.com/tket/user-guide/)
- [t|ket⟩ : A Retargetable Compiler for NISQ Devices](https://arxiv.org/abs/2003.10611)

### 0-2. 今回必要となる python パッケージ
Python Python 3.12.9で動作確認をしています。

|  パッケージ （version） |  概要  |
| :---- | :---- |
|  pytket  |  TKETを利用するためのpython モジュール  ( available for python 3.10 or higher )|
|  pytket-quantinuum  |  Quantinuum Emulatorを利用するためのpytket-extension  |
|  pytket-qiskit  |  IBMQ qiskitを利用するためのpytket-extension  |
|  pytket-braket  |  Amazon Braketを利用するためのpytket-extension  |
|  pytket-qulacs  |  Qulacsを利用するためのpytket-extension  |

<img src="./fig/tket2.png" width="750">

In [ ]:
!pip freeze |grep pytket

環境にインストールされていない場合は、以下のセルの＃を取り除き、インストールしてください。

In [ ]:
#!pip install -U pytket #TKET量子回路の作成、量子回路の最適化をじっこうするためのパッケージ
#!pip install -U pytket-quantinuum #pytketで構成した量子回路をQuantinuumエミュレータで実行するためのパッケージ
#!pip install -U pytket-quantinuum[pecos] #Quantinuumエミュレータのパッケージ
#!pip install -U pytket-qiskit #pytketで構成した量子回路をIBMQデバイス/エミュレータで実行するためのパッケージ
#!pip install -U pytket-braket #pytketで構成した量子回路をAmazon Braketで実行するためのパッケージ
#!pip install -U pytket-qulacs #pytketで構成した量子回路をqulacsで実行するためのパッケージ
#!pip install -U pylatexenc #可視化のためのパッケージ

## 1. 量子回路を作成し、可視化する
ここでは IBMの量子デバイスやシュミレーションを利用できる`qiskit`と`TKET`でベル状態を作成する。

### 1-1. `qiskit`でベル状態を作成し、可視化する
$$ |\Psi\rangle = \frac{1}{\sqrt{2}}(|00\rangle+|11\rangle)$$

In [ ]:
from qiskit import QuantumCircuit

qs_bell = QuantumCircuit(2)
qs_bell.h(0)
qs_bell.cx(0,1)
qs_bell.measure_all()

qs_bell.draw(output='mpl')

### 1-2. `TKET`でベル状態を作成し、可視化する

In [ ]:
from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter

bell = Circuit(2)
bell.H(0).CX(0,1)
bell.measure_all()
render_circuit_jupyter(bell)

## 2. `TKET`の量子回路を量子デバイス/シミュレータで実行

### 2-1. `pytket-braket`でTKET 量子回路をBraketシミュレータやBraketにある量子デバイスで計算

#### A. ローカル環境からBraketにアクセスされている方は、keyフォルダーのaws-keyのアクセスキー等の情報を入力し下記を実行してください

In [ ]:
aws_access_key_id = "your aws_access_key_id"
aws_secret_access_key = "your aws_secret_access_key"
s3_bucket = "your s3_name"
bucket_key = "your bucket_key"

In [ ]:
import boto3
from braket.aws import AwsSession
boto_session = boto3.Session(
    aws_access_key_id= aws_access_key_id,
    aws_secret_access_key= aws_secret_access_key,
    region_name= 'us-east-1'
#    region_name= 'us-west-1'
#    region_name= 'eu-north-1'
)
# us-east-1: IonQ, Simulators
# us-west-1: Rigetti, Simulators
# eu-north-1: IQM
#
aws_session = AwsSession(boto_session=boto_session)

#### B. Sagemaker StudioからBraketにアクセスされている方は下記を実行してください。

In [ ]:
s3_bucket = 'amazon-braket-osaka25'
#bucket_key = 'user25-your_number/pytket'

In [ ]:
import boto3
from braket.aws import AwsSession
boto_session = boto3.Session(
#    region_name= 'us-east-1'
    region_name= 'us-west-1'
#    region_name= 'eu-north-1'
)
# us-east-1: IonQ, Simulators
# us-west-1: Rigetti, Simulators
# eu-north-1: IQM
#
aws_session = AwsSession(boto_session=boto_session)

In [ ]:
device = aws_session.search_devices()

In [ ]:
[dev for dev in device if dev["deviceStatus"]!='RETIRED']

# BraketBackendのオプション
- local: use simulator running on local machine,
            default: False
- local_device: name of local device (ignored if local=False) -- e.g.
            "braket_sv" (default) or "braket_dm".
- device: device name from device ARN (e.g. "Aria-1", "Ankaa-3", ...),
            default: "sv1"
- region: region from device ARN, default: ""
- s3_bucket: name of S3 bucket to store results
- s3_folder: name of folder ("key") in S3 bucket to store results in
- device_type: device type from device ARN (e.g. "qpu"),
            default: "quantum-simulator"
- provider: provider name from device ARN (e.g. "ionq", "rigetti", ...),
            default: "amazon"
- aws_session: braket AwsSession object, to pass credentials in if not
            configured on local machine
- verbatim: use the feature "verbatim-compilation".
            If verbatim-compilation = True, you can execute your circuits composed of
            the primitive gates supported by QPU without any modifications. default: False  

※ IonQのverbatim、IQMの中間測定、フィードフォワードには未対応 (2025/09/02 時点)  
For more details, see  
verbatimについて:  
https://docs.aws.amazon.com/braket/latest/developerguide/braket-constructing-circuit.html#verbatim-compilation  
https://github.com/amazon-braket/amazon-braket-examples/blob/main/examples/braket_features/Verbatim_Compilation.ipynb  
MCMについて:  
https://docs.aws.amazon.com/braket/latest/developerguide/braket-experimental-capabilities.html#braket-access-dynamic-circuits
https://github.com/amazon-braket/amazon-braket-examples/tree/main/examples/experimental_capabilities/dynamic_circuits

In [ ]:
# backend の選択
from pytket.extensions.braket import BraketBackend

# ionQ Aria-1, region: us-east-1
# aws_backend = BraketBackend(device='Aria-1',region = 'us-east-1', s3_bucket=s3_bucket , s3_folder = bucket_key, device_type = 'qpu', provider = 'ionq', aws_session=aws_session, verbatim=False)
# ionQ Aria-2, region: us-east-1
# aws_backend = BraketBackend(device='Aria-2',region = 'us-east-1', s3_bucket=s3_bucket , s3_folder = bucket_key, device_type = 'qpu', provider = 'ionq', aws_session=aws_session, verbatim=False)
# ionQ Forte-1, region: us-east-1
# aws_backend = BraketBackend(device='Forte-1',region = 'us-east-1', s3_bucket=s3_bucket , s3_folder = bucket_key, device_type = 'qpu', provider = 'ionq', aws_session=aws_session, verbatim=False)
# ionQ Forte-Enterprise-1, region: us-east-1
# aws_backend = BraketBackend(device='Forte-Enterprise-1',region = 'us-east-1', s3_bucket=s3_bucket , s3_folder = bucket_key, device_type = 'qpu', provider = 'ionq', aws_session=aws_session, verbatim=False)

# Rigetti Ankaa-3, region: us-west-1
# aws_backend = BraketBackend(device='Ankaa-3',region = 'us-west-1', s3_bucket=s3_bucket , s3_folder = bucket_key, device_type = 'qpu', provider = 'rigetti', aws_session=aws_session, verbatim=False)

# IQM Garnet arn:aws:braket:eu-north-1::device/qpu/iqm/Garnet #region: eu-north-1
# aws_backend = BraketBackend(device='Garnet',region = 'eu-north-1', s3_bucket=s3_bucket , s3_folder = bucket_key, device_type = 'qpu', provider = 'iqm', aws_session=aws_session, verbatim=False)
# IQM Emerald arn:aws:braket:eu-north-1::device/qpu/iqm/ #region: eu-north-1
# aws_backend = BraketBackend(device='Emerald',region = 'eu-north-1', s3_bucket=s3_bucket , s3_folder = bucket_key, device_type = 'qpu', provider = 'iqm', aws_session=aws_session, verbatim=True)

# SV1    sim_arn = 'arn:aws:braket:::device/quantum-simulator/amazon/sv1'     #SV1, region:
aws_backend = BraketBackend(device='sv1', s3_bucket=s3_bucket , s3_folder = bucket_key, device_type = 'quantum-simulator', provider = 'amazon', aws_session=aws_session)
# TN1   sim_arn = 'arn:aws:braket:::device/quantum-simulator/amazon/tn1'    #TN1, region: 
# aws_backend = BraketBackend(device='tn1', s3_bucket=s3_bucket , s3_folder = bucket_key, device_type = 'quantum-simulator', provider = 'amazon', aws_session=aws_session)
# DM1   sim_arn = 'arn:aws:braket:::device/quantum-simulator/amazon/dm1'    #DM1, region: 
# aws_backend = BraketBackend(device='dm1', s3_bucket=s3_bucket , s3_folder = bucket_key, device_type = 'quantum-simulator', provider = 'amazon', aws_session=aws_session)
# Local sv1 simulator
# aws_backend = BraketBackend(local = True)

In [ ]:
aws_backend.backend_info

In [ ]:
from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter
# bell状態の作成
bell = Circuit(2)
bell.H(0).CX(0,1)
bell.measure_all()
render_circuit_jupyter(bell)

# backend に用意されているゲートで量子回路を表現
aws_bell = aws_backend.get_compiled_circuit(bell)
render_circuit_jupyter(aws_bell)

In [ ]:
aws_handle = aws_backend.process_circuit(aws_bell, n_shots =100)
aws_result = aws_backend.get_result(aws_handle)
aws_counts = aws_result.get_counts()
print(aws_counts)
from qiskit.visualization import plot_histogram
plot_histogram(aws_result.get_counts())

### 2-2. `pytket-quantinuum`でTKET 量子回路をQuantinuumのローカルエミュレータで計算

In [ ]:
from pytket.extensions.quantinuum import QuantinuumBackend, QuantinuumAPIOffline
apioffline = QuantinuumAPIOffline()
qntm_backend = QuantinuumBackend(device_name="H1-1LE", api_handler = apioffline)

In [ ]:
qntm_backend.backend_info

In [ ]:
# Compile the circuit using default compilation
qntm_bell = qntm_backend.get_compiled_circuit(bell)

In [ ]:
qntm_handle = qntm_backend.process_circuit(qntm_bell, n_shots=100)

In [ ]:
qntm_result = qntm_backend.get_result(qntm_handle)
qntm_counts = qntm_result.get_counts()
print(qntm_counts)
from qiskit.visualization import plot_histogram
plot_histogram(qntm_result.get_counts())

### 2-3. `pytket-qiskit`でTKET 量子回路をIBMのローカルシミュレータで計算

In [ ]:
from qiskit.visualization import plot_histogram
from pytket.extensions.qiskit import AerBackend
aer_backend = AerBackend()

aer_handle = aer_backend.process_circuit(bell, n_shots =1000)
aer_counts = aer_backend.get_result(aer_handle).get_counts()
plot_histogram(aer_counts)

IBM tokenの設定

In [ ]:
# Replace the placeholders with your actual values
ibm_token = '<your_ibm_token_here>'
inst = '<your_instance_CRN_here>'
from qiskit_ibm_runtime import QiskitRuntimeService
QiskitRuntimeService.save_account(channel="ibm_quantum_platform", token=ibm_token, instance=inst)


In [ ]:
from pytket.extensions.qiskit.backends.config import set_ibmq_config
set_ibmq_config(ibmq_api_token=ibm_token, instance=f"ibm-q/open/main")

In [ ]:
from pytket.extensions.qiskit import IBMQBackend, IBMQEmulatorBackend, AerBackend
from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter

In [ ]:
list_backend=IBMQBackend.available_devices()

In [ ]:
for b in list_backend:
    print(b.device_name)

In [ ]:
# ibm_backend = AerBackend()
# ibm_backend = IBMQBackend("ibm_pittsburgh")
# ibm_backend = IBMQBackend("ibm_pittsburgh")
# AerBackend to emulate the behaviour of IBMQBackend. Requires a valid IBMQ account.
ibm_backend = IBMQEmulatorBackend("ibm_pittsburgh")

In [ ]:
ibm_bell = ibm_backend.get_compiled_circuit(bell)
render_circuit_jupyter(bell)
render_circuit_jupyter(ibm_bell)

In [ ]:
handle = ibm_backend.process_circuit(ibm_bell, n_shots =1000)
result_counts = ibm_backend.get_result(handle).get_counts()
plot_histogram(result_counts)

### 2-4. `pytket-qulacs`でTKET 量子回路をQulacsシミュレータで計算

In [ ]:
from pytket.extensions.qulacs import QulacsBackend
qulacs_backend = QulacsBackend()

qulacs_handle = qulacs_backend.process_circuit(bell, n_shots =1000)
qulacs_counts = qulacs_backend.get_result(qulacs_handle).get_counts()
plot_histogram(qulacs_counts)

#### GPU上でQulacsを利用している場合にも対応している


In [ ]:
#from pytket.extensions.qulacs import QulacsGPUBackend
#backend = QulacsGPUBackend()

#handle = backend.process_circuit(bell, n_shots =1000)
#result_counts = backend.get_result(handle).get_counts()
#plot_histogram(result_counts)

詳しくは
https://docs.quantinuum.com/tket/extensions/pytket-qulacs/#gpu-acceleration
を参照ください

## 3. 量子回路の変換
pytketでは
- qiskitで記述した量子回路(`qiskit.QuantumCircuit`)からTKETの量子回路のクラスに変換が可能
- TKETで記述した量子回路からqiskitの量子回路(`qiskit.QuantumCircuit`)のクラスに変換が可能
- TKETで記述した量子回路と他の量子プログラミング言語(一部)での量子回路の交換が可能

参照：[pytket-extensions](https://docs.quantinuum.com/tket/api-docs/extensions.html) 

### 3-1. `qiskit`の量子回路から`TKET`の量子回路に変換

In [ ]:
from pytket.extensions.qiskit import qiskit_to_tk

In [ ]:
bell2 = qiskit_to_tk(qs_bell)
bell2

In [ ]:
render_circuit_jupyter(bell2)

### 3-2. `TKET`の量子回路から`qiskit`の量子回路に変換

In [ ]:
from pytket.extensions.qiskit import tk_to_qiskit

In [ ]:
qs_bell2 = tk_to_qiskit(bell)
qs_bell2

In [ ]:
qs_bell2.draw('mpl')

### 3-3. `TKET`の量子回路から`braket`の量子回路に変換

In [ ]:
from pytket.extensions.braket import tk_to_braket
aws_bell2 = tk_to_braket(bell)[0]
print(aws_bell2)

### 3-4. `braket`の量子回路から`TKET`の量子回路に変換

In [ ]:
from pytket.extensions.braket import braket_to_tk
bell3 = braket_to_tk(aws_bell2)
render_circuit_jupyter(bell3)

# 弊社Quantinuumのご紹介
- Website（ 英語 ）： https://www.quantinuum.com/
- ウェブサイト（ 日本語 ）： https://quantinuum.co.jp/
- Press Releases（ 英語 ）： https://www.quantinuum.com/news/news#press-release
- ニュース（ 日本語 ）： https://quantinuum.co.jp/news/
- X（ 日本語 ）： https://x.com/quantinuum_jp
- 採用情報（ 英語 ）：https://www.quantinuum.com/careers
